In [1]:
import numpy as np
import time
import random
import psutil

In [2]:
# 获取数据信息
f = open("ca-AstroPh.txt", 'r')
firstline = f.readline()
f.close()
print(firstline)

% 18771 198050



In [3]:
# 定义全局变量
N = 18771  # 节点数目
r = 3
b = 3000
mh_n = r*b
all_mhs = range(mh_n)
all_nodes = range(1, N+1)
TOP_K = 10

In [4]:
# 转换成邻接表
A = {}
f = open("ca-AstroPh.txt", 'r')
f.readline()
while True:
    try:
        line = f.readline().strip().split(' ')  # 去掉回车符并分割
        i = int(line[0])
        j = int(line[1])
        try:
            A[i].add(j)
        except Exception:
            A[i] = set([])
            A[i].add(j)
        try:
            A[j].add(i)
        except Exception:
            A[j] = set([])
            A[j].add(i)
    except ValueError:
        break
f.close()
for i in all_nodes:
    A[i] = list(A[i])

In [5]:
mhtab = np.zeros((mh_n, N+1), dtype='int32')
start = time.time()
for i in all_mhs:
    for j in all_nodes:
        np.random.shuffle(A[j])
        mhtab[i, j] = A[j][0]
end1 = time.time()
print("最小哈希签名矩阵构建时间：", end1 - start, "s")
np.save("mhtab", mhtab)

最小哈希签名矩阵构建时间： 329.1273694038391 s


In [6]:
# 映射到桶
bucket = {}  
for i in range(b):   # i表示第i组
    for k in all_nodes:
        sign = mhtab[i*r:(i+1)*r, k]   # 数组切片表示签名
        sign = str([s for s in sign])
        try:
            bucket[sign].add(k)
        except Exception:
            bucket[sign] = set([])
            bucket[sign].add(k)
end2 = time.time()
print("桶映射时间：", end2 - end1, "s")
print("索引时间：", end2 - start, "s")

桶映射时间： 296.23503160476685 s
索引时间： 625.362401008606 s


In [7]:
def meminfo():    
    # 获取当前进程的内存信息
    process = psutil.Process()
    memory_info = process.memory_info()
    # 获取内存占用大小（以字节为单位）
    memory_usage = memory_info.rss
    # 打印内存占用大小（以Mb为单位）
    print("当前程序占用的内存：", memory_usage/1024/1024, "MB")
    # 获取系统内存信息
    memory_info = psutil.virtual_memory()
    # 获取内存使用率（以百分比表示）
    memory_usage_percent = memory_info.percent
    # 打印内存使用率
    print("当前程序的空间使用率：", memory_usage_percent, "%")

In [8]:
meminfo()

当前程序占用的内存： 7903.2421875 MB
当前程序的空间使用率： 98.4 %


In [9]:
def Jaccard(n1, n2):  # 计算两个结点的Jaccard相似度
    # n1, n2: set([1,3,5])
    mu = len(n1 & n2)
    de = len(n1 | n2)
    return mu/de


def query(node):
    # 候选集
    candi = set([])
    for i in range(b):
         # 获取签名并更新候选集
        sign = mhtab[i*r:(i+1)*r, node]  # 数组切片表示签名
        sign = str([s for s in sign])
        candi |= (bucket[sign])
    candi.remove(node)
    # 计算与候选集中Jaccard相似度最大的top10
    Jlist = {}  # 保存所有Jaccard
    n1 = set(A[node])
    for c in candi:
        n2 = set(A[c])
        Jlist[c] = Jaccard(n1, n2)
    res = sorted(Jlist.items(), key=lambda x: x[1], reverse=True)[:TOP_K]
    return res

In [10]:
start = time.time()
print(query(1))
end = time.time()
print("查询时间:{} s".format(end-start))

[(17, 0.23333333333333334), (40, 0.14130434782608695), (63, 0.12790697674418605), (75, 0.11956521739130435), (56, 0.09411764705882353), (42, 0.07894736842105263), (19, 0.07407407407407407), (61, 0.07086614173228346), (2, 0.0707070707070707), (29, 0.06593406593406594)]
查询时间:0.024412870407104492 s


In [11]:
meminfo()

当前程序占用的内存： 7899.359375 MB
当前程序的空间使用率： 98.9 %


In [12]:
def accurate(node):
    Jaccards = {}
    x = set(A[node])
    all_nodes_set = set([n for n in all_nodes])
    all_nodes_set.remove(node)
    for i in all_nodes_set:
        y = set(A[i])
        Jaccards[i] = Jaccard(x, y)
    accurate_res = sorted(Jaccards.items(), key=lambda x: x[1], reverse=True)[: TOP_K]
    my_res = query(node)
    mu = len(set([r[0] for r in accurate_res]) & set([r[0] for r in my_res]))
    return mu/TOP_K

In [13]:
sum_ = 0
for i in range(100):
    node = random.randint(1, N)
    a = accurate(node)
    sum_ += a
print("准确率：", sum_/100)

准确率： 0.8620000000000002
